# Slackの会話データのベクタライズモデル（Doc2Vec）を学習させる

---

## 1. はじめに

本ノートブックでは、Slack上の会話データをベクトル化する機械学習モデルの構築を行います。

機械学習のアルゴリズムには、Doc2Vecを利用します。

**本 Notebook は Google Colab 上で実行することを前提としています。**


## 2. 事前に準備していること

本ノートブックの作成前に、すでに以下のことが完了していますので、ノートブック内では本処理を実行するコードやその解説は述べません。


- Slackの会話情報の抽出と整形
- 上記整形済みデータをGoogleDriveにアップロード（`dataset.json`）



## 3. 本ノートブックの目的

本ノートブックの目的は、以下の３つの処理を実行すること、及びその解説をすることです。

- Doc2Vecモデルの学習
- 学習済みモデルの動作確認
- 学習済みモデルの保存


---

## 4. 処理フロー

1. GoogleDriveのマウント
2. 必要なパッケージのインストール／インポート
3. データセットのロード
4. データセットの確認
5. Doc2Vecモデルの学習
6. 学習済みモデルの動作確認
7. 学習済みモデルの保存

<br>

### 4.1. 処理ごとの目的と入出力一覧

|項番|目的|入力|出力|
|:-:|:--|:--|:--|
|1|GoogleDriveとColabNotebookを接続<br>（データセット読込の準備）|−|−|
|2|学習、検証処理などに利用するパッケージを使用可能な状態にする|−|−|
|3|データセットをメモリ上で操作可能な状態にする|ファイルパス|データセット|
|4|学習後の手戻りを抑制するために、<br>データセットに予期せぬ誤りがないか確認する|データセット|データセットの中身（標準出力）|
|5|学習させたいデータセットにモデルを適応させる|未学習モデル|学習済みモデル|
|6|学習済みモデルが予期した動作をしているか簡易確認を行い、<br>後工程の手戻りを抑制する|学習済みモデル<br>/検証用デモデータ|計算結果|
|7|学習結果を再利用できるようにファイル化する|−|学習済みモデルファイル|

<br>

---


## 5. 処理詳細

### 5.1. GoogleDriveのマウント

GoogleColabにデフォルトで搭載されている `google.colab` パッケージを使って、自身のGoogleDriveをマウントします。

これにより、GoogleDriveをローカルファイルシステムのように扱えるようになります。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 5.2. 必要なパッケージのインストール／インポート

今回のDoc2Vecモデルの学習に必要なパッケージをまとめて、インストール／インポートします。

今回利用するパッケージとその用途を下記にまとめます。

GoogleColabにデフォルトで組み込まれていないパッケージは、明示的に `pip install ...` を実行します。


|分類|パッケージ名|用途|
|:-:|---|:--|
|標準ライブラリ|json|データセット（JSON形式）の操作の為|
||smart_open|大規模データのファイルストリーミング用（通常データセットは大規模である場合が多い）|
|3rdpartyライブラリ|numpy|COS類似度算出処理のため|
||gensim|Doc2Vecを使用するため|


In [ ]:
!pip install gensim

In [ ]:
import json
import smart_open

import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

### 5.3. データセットのロード

GoogleDriveに格納しているデータセットをロードします。

`dataset.json`の構造は以下のようになっています。

```json
[
  {
    "tag": 0,
    "text": ["word0", "word1", "word2"...]
  },
  {
    "tag": 1,
    "text": ["word0", "word1", "word2"...]
  },
  :
]
```

これらをgensimのTggedDocumentインスタンスに格納しています。


【参考】：データセットのTggedDocument形式へのロードは、[gensimの公式ドキュメント](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html?highlight=dataset#define-a-function-to-read-and-preprocess-text)を参照して実装しました。


In [ ]:
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="utf-8") as f:
        dataset = json.load(f)
        for doc in dataset:
          doc_tag = int(doc['tag'])
          doc_text = doc['text']
          if tokens_only:
            yield doc_text
          else:
            # For training data, add tags
            yield TaggedDocument(doc_text, [doc_tag])

In [ ]:
root_dir_path = '/content/drive/My Drive/app/data/portfolio-word-embedding/'

In [ ]:
train_corpus = list(read_corpus(root_dir_path + 'dataset.json'))

### 5.4. データセットの確認

In [ ]:
# see coupus
train_corpus[:10]

### 5.5. Doc2Vecモデルの学習

ハイパーパラメータを設定して、モデルを学習させます。

ハイパーパラメータの設定値、その理由を簡単に述べます。

- vector_size
  - 算出するベクトルの次元を表します
  - 400：[Doc2Vecの論文](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)と同様に設定しました
- window
  - 一回にいくつの単語をネットワークに読み込むか、という値です
  - 8：[Doc2Vecの論文](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)に依れば、多くのアプリケーションでwindow sizeが8~12の範囲が良い結果をもたらした、との説明に基づいて設定しました。
- min_count
  - 1語と認識するための最低文字数です
  - 1：日本語は1文字で1語を表す（「が」など）ことがあるので1に設定しました
- epochs
  - 1つのデータを何回繰り返し使うかという値です
  - 40：いくつかの実装例を参考に設定しました

In [ ]:
model = Doc2Vec(vector_size=400, window=8, min_count=1, epochs=40)
model

In [ ]:
model.build_vocab(train_corpus)

In [ ]:
model.corpus_count

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

### 5.6. 学習済みモデルの動作確認

デモ用データを入力して、文章のベクトル化が期待通りに動いているか、簡単に確認します。

文章同士が似ていれば似ているほど、ベクトル同士の類似度が高いことを確認します。

In [ ]:
# COS類似度
def cos_similarity(_x: list, _y: list) -> float:
    vx = np.array(_x)
    vy = np.array(_y)
    return np.dot(vx, vy) / (np.linalg.norm(vx) * np.linalg.norm(vy))

In [ ]:
v_py1_ja4 = model.infer_vector(['python', 'java', 'java', 'java', 'java'])
v_py3_ja2 = model.infer_vector(['python', 'python', 'python', 'java', 'java'])
v_py4_ja1 = model.infer_vector(['python', 'python', 'python', 'python', 'java'])

v_py5_ja0 = model.infer_vector(['python', 'python', 'python', 'python', 'python'])


print('「python x 5」文ベクトルとのCOS類似度')
print('===============================')
print('python x 1, java x 4 >> {}'.format(cos_similarity(v_py5_ja0, v_py1_ja4)))
print('python x 3, java x 2 >> {}'.format(cos_similarity(v_py5_ja0, v_py3_ja2)))
print('python x 4, java x 1 >> {}'.format(cos_similarity(v_py5_ja0, v_py4_ja1)))

### 5.7. 学習済みモデルの保存

In [ ]:
# save
import pickle
with open(root_dir_path + 'trained_doc2vec.model.pkl', 'wb') as f:
    pickle.dump(model, f)

## 6. 【補足】今後の改善点

[Doc2Vecによる文書ベクトル推論の安定化について | Sansan Builders Blog](https://buildersbox.corp-sansan.com/entry/2019/04/10/110000#Doc2Vec%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E6%9B%B8%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E6%8E%A8%E8%AB%96%E3%81%AE%E5%95%8F%E9%A1%8C%E7%82%B9)

上記参考記事の通り、Doc2Vecには「極めて短い文」のベクトル化を苦手とする側面があります。

今回のケースだと、質問文のベクトル化の際、短い文のベクトル化を実行するケースが起こり得る（例：機械学習とは？）ので、対策を打つ必要があると考えています。

具体的な対策案としては、今の所以下の２つを考えています。

1. Doc2Vec以外のモデルを使う
2. 入力文字数に制限（最低入力文字数、単語数）を設定する

２番目の対策は、プロダクト開発フェーズで実施すべき項目であることと、本来の課題解決の目的に合致していないことから優先度は低く、１番目の対策を実施すべきと考えています。

１番目の対策をより具体的にするならば、会話情報の名詞のみ対象として、Word2Vecによるベクトル化を検討しています。

この場合、１ユーザーを高次元の１ベクトルで表現するのではなく、１ユーザーをより低次元の複数のベクトルで表現することになると考えています。


![img](https://drive.google.com/uc?export=view&id=13OT9iqbhO54hZlTSD6Z8koWHX12Tsfsw)
